In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split,col,avg

In [2]:
spark = SparkSession.builder.appName('PySparkTitanikJob').getOrCreate()

In [3]:
spark

In [9]:
df = spark.read.option('header','true').csv('train.csv')

In [10]:
df.show(2)

+-----------+--------+------+--------------------+------+---+-----+-----+---------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+---------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+---+-----+-----+---------+-------+-----+--------+
only showing top 2 rows



In [15]:
df.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'string'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [16]:
df.describe(['Sex', 'Age']).show()

+-------+------+------------------+
|summary|   Sex|               Age|
+-------+------+------------------+
|  count|   891|               714|
|   mean|  null| 29.69911764705882|
| stddev|  null|14.526497332334035|
|    min|female|              0.42|
|    max|  male|                 9|
+-------+------+------------------+



In [17]:
df.count()

891

In [18]:
df = df.drop('Ticket', 'Name', 'Fare','Cabin')

In [19]:
df.show()

+-----------+--------+------+------+----+-----+-----+--------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|Embarked|
+-----------+--------+------+------+----+-----+-----+--------+
|          1|       0|     3|  male|  22|    1|    0|       S|
|          2|       1|     1|female|  38|    1|    0|       C|
|          3|       1|     3|female|  26|    0|    0|       S|
|          4|       1|     1|female|  35|    1|    0|       S|
|          5|       0|     3|  male|  35|    0|    0|       S|
|          6|       0|     3|  male|null|    0|    0|       Q|
|          7|       0|     1|  male|  54|    0|    0|       S|
|          8|       0|     3|  male|   2|    3|    1|       S|
|          9|       1|     3|female|  27|    0|    2|       S|
|         10|       1|     2|female|  14|    1|    0|       C|
|         11|       1|     3|female|   4|    1|    1|       S|
|         12|       1|     1|female|  58|    0|    0|       S|
|         13|       0|     3|  male|  20|    0|    0|  

In [28]:
df = df.withColumn('FamilySize', col('SibSp') + col('Parch') + 1)

In [29]:
df.show(4)

+-----------+--------+------+------+---+-----+-----+--------+----------+
|PassengerId|Survived|Pclass|   Sex|Age|SibSp|Parch|Embarked|FamilySize|
+-----------+--------+------+------+---+-----+-----+--------+----------+
|          1|       0|     3|  male| 22|    1|    0|       S|       2.0|
|          2|       1|     1|female| 38|    1|    0|       C|       2.0|
|          3|       1|     3|female| 26|    0|    0|       S|       1.0|
|          4|       1|     1|female| 35|    1|    0|       S|       2.0|
+-----------+--------+------+------+---+-----+-----+--------+----------+
only showing top 4 rows



In [30]:
df.where(col('Age').isNull()).count()

177

In [33]:
avg_age = df.select(avg(col('Age'))).collect()[0][0]

In [36]:
ndf = df.fillna({'Age': avg_age})

In [37]:
ndf.show(5)

+-----------+--------+------+------+---+-----+-----+--------+----------+
|PassengerId|Survived|Pclass|   Sex|Age|SibSp|Parch|Embarked|FamilySize|
+-----------+--------+------+------+---+-----+-----+--------+----------+
|          1|       0|     3|  male| 22|    1|    0|       S|       2.0|
|          2|       1|     1|female| 38|    1|    0|       C|       2.0|
|          3|       1|     3|female| 26|    0|    0|       S|       1.0|
|          4|       1|     1|female| 35|    1|    0|       S|       2.0|
|          5|       0|     3|  male| 35|    0|    0|       S|       1.0|
+-----------+--------+------+------+---+-----+-----+--------+----------+
only showing top 5 rows



In [43]:
ndf[['Sex']].distinct().show()

+------+
|   Sex|
+------+
|female|
|  male|
+------+



In [44]:
ndf = ndf.withColumn('M', col('Sex') == 'male')

In [45]:
ndf = ndf.withColumn('W', col('Sex') == 'female')

In [46]:
ndf.show(2)

+-----------+--------+------+------+---+-----+-----+--------+----------+-----+-----+
|PassengerId|Survived|Pclass|   Sex|Age|SibSp|Parch|Embarked|FamilySize|    M|    W|
+-----------+--------+------+------+---+-----+-----+--------+----------+-----+-----+
|          1|       0|     3|  male| 22|    1|    0|       S|       2.0| true|false|
|          2|       1|     1|female| 38|    1|    0|       C|       2.0|false| true|
+-----------+--------+------+------+---+-----+-----+--------+----------+-----+-----+
only showing top 2 rows



In [47]:
ndf = ndf.drop('Sex')

In [48]:
ndf.show(5)

+-----------+--------+------+---+-----+-----+--------+----------+-----+-----+
|PassengerId|Survived|Pclass|Age|SibSp|Parch|Embarked|FamilySize|    M|    W|
+-----------+--------+------+---+-----+-----+--------+----------+-----+-----+
|          1|       0|     3| 22|    1|    0|       S|       2.0| true|false|
|          2|       1|     1| 38|    1|    0|       C|       2.0|false| true|
|          3|       1|     3| 26|    0|    0|       S|       1.0|false| true|
|          4|       1|     1| 35|    1|    0|       S|       2.0|false| true|
|          5|       0|     3| 35|    0|    0|       S|       1.0| true|false|
+-----------+--------+------+---+-----+-----+--------+----------+-----+-----+
only showing top 5 rows



In [51]:
ndf.coalesce(1).write.option('header','true').csv('clear_data')